# Speckle Tracking

In [40]:
from pathlib import Path
from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *

set_jupyter_backend("pythreejs")

sample_file_ideal = Path("../../_static/sample_files/ellipsoid.stl")
lv_ideal = LV.from_file(sample_file_ideal) 

lv_ideal.identify_regions(geo_type=LV_GEO_TYPES.IDEAL,
                          apex_base_args=dict(ab_ql=0.05, ab_qh=0.97),
                          recompute_apex_base=True
                          )

import numpy as np
import logging

epi_spks = []
for i, a in enumerate(np.linspace(0, np.pi, 6, endpoint=False)):
    
    spk = lv_ideal.create_speckles(
        collection="long-6",
        group="endo",
        name=str(i),
        from_nodeset=LV_SURFS.ENDO,
        exclude_nodeset=LV_SURFS.BASE, # does not afect ideal case
        d=1.75,
        k=0.5,
        normal_to=[np.cos(a),np.sin(a),0.0],
        n_subsets=6,
        subsets_criteria="z2",
        cluster_criteria="z2",
        n_clusters=8,
        t=0.0,
        kmin=-1,
        kmax=0.95,
        log_level=logging.WARN,
    )

    spk = lv_ideal.create_speckles(
        collection="long-6",
        group="epi",
        name=str(i),
        from_nodeset=LV_SURFS.EPI,
        exclude_nodeset=LV_SURFS.BASE, # does not afect ideal case
        d=2.4,
        k=0.5,
        normal_to=[np.cos(a),np.sin(a),0.0],
        n_subsets=6,
        subsets_criteria="z2",
        cluster_criteria="z2",
        n_clusters=10,
        t=0.0,
        kmin=-1,
        kmax=0.95,
        log_level=logging.INFO,
    )
    epi_spks.append(spk)

INFO:create_speckles:Speckle: name: 0, group: epi, collection: long-6
INFO:create_speckles:Speckle: name: 1, group: epi, collection: long-6
INFO:create_speckles:Speckle: name: 2, group: epi, collection: long-6
INFO:create_speckles:Speckle: name: 3, group: epi, collection: long-6
INFO:create_speckles:Speckle: name: 4, group: epi, collection: long-6
INFO:create_speckles:Speckle: name: 5, group: epi, collection: long-6


# What are Speckles?

In [41]:
lv_ideal.plot_speckles(epi_spks[1], add_centers=True, cmap="tab10", categories=True)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(position=(50000.000000000015, 70710.6…

In [42]:
lv_ideal.plot_speckles(spk, cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(position=(50000.000000000015, 70710.6…